In [3]:
from google.cloud import bigquery
import pandas as pd
import requests
import datetime
import re
import json
from datetime import date, timedelta
#client = bigquery.Client(location="us-central1")
client = bigquery.Client(location="US")
#print("Client creating using default project: {}".format(client.project))
pd.set_option('display.width', 1000)

In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName("GCP_Storage_Cost").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/28 21:04:21 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/03/28 21:04:21 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/03/28 21:04:22 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/03/28 21:04:22 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [7]:
current_date = datetime.datetime.today()

dias_atras = 4
#dias_atras = int(dias_atras)

str_day_process = (current_date - timedelta(days = dias_atras)).strftime("%Y-%m-%d")
str_day_process_dt = pd.to_datetime(str_day_process)
str_lastday = (str_day_process_dt - timedelta(days = 1)).strftime("%Y-%m-%d")
str_today = (str_day_process_dt + timedelta(days = 1)).strftime("%Y-%m-%d")

#STR_YEARMONTH = (current_date - timedelta(days = dias_atras)).strftime("%Y%m")
#str_day_process = "2022-07-25"
#STR_YEARMONTH = "201908"
print("Dia Analizado --> ",str_day_process," ; ","extraido de :",str_lastday," - ",str_today)

Dia Analizado -->  2023-03-24  ;  extraido de : 2023-03-23  -  2023-03-25


### YT1 and VT1 Storage Cost Extraction - GCP

In [8]:
environment = 'YT1'

In [10]:
if environment == 'YT1':
    project_number = '260718309650'

if environment == 'VT1':
    project_number = '884190693971'

In [11]:
query = """
 SELECT service.description as description_service,sku.description as description_sku,usage_start_time, project.id as id_project, cost, 
 usage.amount as usage_amount, usage.unit as usage_unit, credits, cost_type, resource.name as resource_name, resource.global_name as resource_global_name
 FROM `saas-analytics-io.finance.gcp_billing_export_resource_v1_01FE23_B1D2D8_10D34D` 
 WHERE JSON_VALUE(project.number) = '%s' 
and usage_start_time between '%s 00:00:00' and '%s 23:59:59'
-- and date(usage_start_time) between '2022-07-01' and '2022-07-25'
"""%(project_number,str_lastday, str_today)

query_job = client.query(
  query,
    # Location must match that of the dataset(s) referenced in the query.
  location = "US",
)  # API request - starts the query
print(query)
GCP_cost_table = query_job.to_dataframe()
GCP_cost_table.head()


 SELECT service.description as description_service,sku.description as description_sku,usage_start_time, project.id as id_project, cost, 
 usage.amount as usage_amount, usage.unit as usage_unit, credits, cost_type, resource.name as resource_name, resource.global_name as resource_global_name
 FROM `saas-analytics-io.finance.gcp_billing_export_resource_v1_01FE23_B1D2D8_10D34D` 
 WHERE JSON_VALUE(project.number) = '260718309650' 
and usage_start_time between '2023-03-23 00:00:00' and '2023-03-25 23:59:59'
-- and date(usage_start_time) between '2022-07-01' and '2022-07-25'



,description_service,description_sku,usage_start_time,id_project,cost,usage_amount,usage_unit,credits,cost_type,resource_name,resource_global_name
0,BigQuery,Active Logical Storage (us-central1),2023-03-23 01:00:00+00:00,saas-mojixretail-io,0.103913,1.297578e+16,byte-seconds,[],regular,None,None
1,BigQuery,Active Logical Storage (us-central1),2023-03-23 10:00:00+00:00,saas-mojixretail-io,0.105907,1.322488e+16,byte-seconds,[],regular,None,None
2,BigQuery,Active Logical Storage (us-central1),2023-03-23 05:00:00+00:00,saas-mojixretail-io,0.104188,1.301012e+16,byte-seconds,[],regular,None,None
3,BigQuery,Active Logical Storage (us-central1),2023-03-23 09:00:00+00:00,saas-mojixretail-io,0.105777,1.320861e+16,byte-seconds,[],regular,None,None
4,BigQuery,Active Logical Storage (us-central1),2023-03-23 07:00:00+00:00,saas-mojixretail-io,0.105005,1.311223e+16,byte-seconds,[],regular,None,None


In [12]:
GCP_cost_table['credits'] = GCP_cost_table['credits'].astype(str).str.replace("[","")
GCP_cost_table['credits'] = GCP_cost_table['credits'].astype(str).str.replace("]","")
GCP_cost_table['credits'] = GCP_cost_table['credits'].astype(str).str.replace("","")
try:
    GCP_cost_table.loc[GCP_cost_table['credits'].str.contains('1|2|3|4|5|6|7|8|9|0'),'credits_1'] = GCP_cost_table['credits'].str.split('\n', expand=True)[0]
    GCP_cost_table.loc[GCP_cost_table['credits_1'].isnull(),'credits_1']=''
    json_cols = ['credits_1']
    try:
        GCP_cost_table.loc[GCP_cost_table['credits'].str.contains('1|2|3|4|5|6|7|8|9|0'),'credits_2'] = GCP_cost_table['credits'].str.split('\n', expand=True)[1]
        GCP_cost_table.loc[GCP_cost_table['credits_2'].isnull(),'credits_2']=''
        json_cols = ['credits_1','credits_2']
    except:
        print ('2')
        json_cols = ['credits_1']
except:
    GCP_cost_table.loc[GCP_cost_table['credits'].str.contains('1|2|3|4|5|6|7|8|9|0'),'credits_1'] = GCP_cost_table['credits'][0]
    GCP_cost_table.loc[GCP_cost_table['credits_1'].isnull(),'credits_1']=''
    json_cols = ['credits_1']

#sku = GCP_cost_table['sku'].values.tolist()
#service = GCP_cost_table['service'].values.tolist()

#json_cols = ['service', 'sku','usage','credits']
for column in json_cols:
    #c_load = test[column].apply(json.loads)
    print(column)
    
    if((column=='credits_1')|(column== 'credits_2')):
        GCP_cost_table.loc[~(GCP_cost_table[column].str.contains('1|2|3|4|5|6|7|8|9|0')),column] = "{'name':'','amount':0,'full_name':'','id':'', 'type':''}"
        t_er = list(GCP_cost_table[column])
        t_er2 = json.dumps(t_er)
        t_er2 = t_er2.replace('"',"").replace("'",'"')
        if column == 'credits_1':
            df_credits = pd.DataFrame.from_records(list(eval(t_er2)), columns=['name', 'amount','full_name','id','type'])
            df_credits.rename(columns ={'amount':'amount_credits_1','id':'id_credits_1','name':'name_credits_1'
                                 ,'full_name':'full_name_credits_1', 'type':'type_credits_1'},inplace = True)
            GCP_cost_table = GCP_cost_table.join(df_credits)
        if column == 'credits_2':
            df_credits_2 = pd.DataFrame.from_records(list(eval(t_er2)), columns=['name', 'amount','full_name','id','type'])
            df_credits_2.rename(columns ={'amount':'amount_credits_2','id':'id_credits_2','name':'name_credits_2'
                                 ,'full_name':'full_name_credits_2', 'type':'type_credits_2'},inplace = True)
            GCP_cost_table = GCP_cost_table.join(df_credits_2)

        GCP_cost_table = GCP_cost_table.drop(column , axis=1)

/tmp/ipykernel_2536/1321368945.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  GCP_cost_table['credits'] = GCP_cost_table['credits'].astype(str).str.replace("[","")
/tmp/ipykernel_2536/1321368945.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  GCP_cost_table['credits'] = GCP_cost_table['credits'].astype(str).str.replace("]","")


2
credits_1


In [13]:
GCP_cost_table.loc[(~(GCP_cost_table['description_sku'].str.contains('SSD'))),'compute_cost']=  GCP_cost_table['cost']
GCP_cost_table.loc[(GCP_cost_table['description_sku'].str.contains('SSD')),'storage_cost']=  GCP_cost_table['cost']
try:    
    GCP_cost_table.loc[:,'total_after_discounts']=  GCP_cost_table['cost'] + GCP_cost_table['amount_credits_1']+ GCP_cost_table['amount_credits_2']
    GCP_cost_table.loc[:,'amount_credits']= GCP_cost_table['amount_credits_1']+ GCP_cost_table['amount_credits_2']
    GCP_cost_table.loc[(~(GCP_cost_table['description_sku'].str.contains('SSD'))),'compute_discount']=  GCP_cost_table['amount_credits_1']+GCP_cost_table['amount_credits_2']
    GCP_cost_table.loc[(GCP_cost_table['description_sku'].str.contains('SSD')),'storage_discount']=  GCP_cost_table['amount_credits_1']+GCP_cost_table['amount_credits_2']
    GCP_cost_table.loc[(~(GCP_cost_table['description_sku'].str.contains('SSD'))),'total_compute_after_discount']=  GCP_cost_table['cost'] + GCP_cost_table['amount_credits_1']+ GCP_cost_table['amount_credits_2']
except:
    GCP_cost_table.loc[:,'total_after_discounts']=  GCP_cost_table['cost'] + GCP_cost_table['amount_credits_1']
    GCP_cost_table.loc[:,'amount_credits']= GCP_cost_table['amount_credits_1']
    GCP_cost_table.loc[(~(GCP_cost_table['description_sku'].str.contains('SSD'))),'compute_discount']=  GCP_cost_table['amount_credits_1']
    GCP_cost_table.loc[(GCP_cost_table['description_sku'].str.contains('SSD')),'storage_discount']=  GCP_cost_table['amount_credits_1']
    GCP_cost_table.loc[(~(GCP_cost_table['description_sku'].str.contains('SSD'))),'total_compute_after_discount']=  GCP_cost_table['cost'] + GCP_cost_table['amount_credits_1'] 

    GCP_cost_table['compute_cost'].fillna(0,inplace=True)
    GCP_cost_table['storage_cost'].fillna(0,inplace=True)
    GCP_cost_table['storage_discount'].fillna(0,inplace=True)

In [14]:
GCP_cost_table.head()

,description_service,description_sku,usage_start_time,id_project,cost,usage_amount,usage_unit,credits,cost_type,resource_name,...,full_name_credits_1,id_credits_1,type_credits_1,compute_cost,storage_cost,total_after_discounts,amount_credits,compute_discount,storage_discount,total_compute_after_discount
0,BigQuery,Active Logical Storage (us-central1),2023-03-23 01:00:00+00:00,saas-mojixretail-io,0.103913,1.297578e+16,byte-seconds,,regular,None,...,,,,0.103913,0.0,0.103913,0.0,0.0,0.0,0.103913
1,BigQuery,Active Logical Storage (us-central1),2023-03-23 10:00:00+00:00,saas-mojixretail-io,0.105907,1.322488e+16,byte-seconds,,regular,None,...,,,,0.105907,0.0,0.105907,0.0,0.0,0.0,0.105907
2,BigQuery,Active Logical Storage (us-central1),2023-03-23 05:00:00+00:00,saas-mojixretail-io,0.104188,1.301012e+16,byte-seconds,,regular,None,...,,,,0.104188,0.0,0.104188,0.0,0.0,0.0,0.104188
3,BigQuery,Active Logical Storage (us-central1),2023-03-23 09:00:00+00:00,saas-mojixretail-io,0.105777,1.320861e+16,byte-seconds,,regular,None,...,,,,0.105777,0.0,0.105777,0.0,0.0,0.0,0.105777
4,BigQuery,Active Logical Storage (us-central1),2023-03-23 07:00:00+00:00,saas-mojixretail-io,0.105005,1.311223e+16,byte-seconds,,regular,None,...,,,,0.105005,0.0,0.105005,0.0,0.0,0.0,0.105005


In [15]:
GCP_cost_table.loc[:,'usage_start_time_LA']=GCP_cost_table['usage_start_time'].dt.tz_convert('America/Los_Angeles')

In [16]:
GCP_cost_table.loc[:,'date_LA'] = GCP_cost_table['usage_start_time_LA'].dt.date

In [17]:
GCP_cost_table.tail()

,description_service,description_sku,usage_start_time,id_project,cost,usage_amount,usage_unit,credits,cost_type,resource_name,...,type_credits_1,compute_cost,storage_cost,total_after_discounts,amount_credits,compute_discount,storage_discount,total_compute_after_discount,usage_start_time_LA,date_LA
512746,Compute Engine,Network Internet Egress from Americas to Austr...,2023-03-24 21:00:00+00:00,saas-mojixretail-io,0.000001,5494.0,bytes,,regular,gke-centralus-saas-k8s-mojix-io-moinj-c06f5045...,...,,0.000001,0.0,0.000001,0.0,0.0,0.0,0.000001,2023-03-24 14:00:00-07:00,2023-03-24
512747,Compute Engine,Network Internet Egress from Americas to Austr...,2023-03-24 16:00:00+00:00,saas-mojixretail-io,0.000001,5682.0,bytes,,regular,gke-centralus-saas-k8s-mojix-io-moinj-c06f5045...,...,,0.000001,0.0,0.000001,0.0,0.0,0.0,0.000001,2023-03-24 09:00:00-07:00,2023-03-24
512748,Compute Engine,Network Internet Egress from Americas to Austr...,2023-03-24 20:00:00+00:00,saas-mojixretail-io,0.000001,5494.0,bytes,,regular,gke-centralus-saas-k8s-mojix-io-moinj-c06f5045...,...,,0.000001,0.0,0.000001,0.0,0.0,0.0,0.000001,2023-03-24 13:00:00-07:00,2023-03-24
512749,Compute Engine,Network Internet Egress from Americas to Austr...,2023-03-25 10:00:00+00:00,saas-mojixretail-io,0.000000,1260.0,bytes,,regular,gke-centralus-saas-k8s-mojix-io-moinj-c06f5045...,...,,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,2023-03-25 03:00:00-07:00,2023-03-25
512750,Compute Engine,Network Internet Egress from Americas to Austr...,2023-03-25 03:00:00+00:00,saas-mojixretail-io,0.000000,40.0,bytes,,regular,gke-centralus-saas-k8s-mojix-io-moinj-c06f5045...,...,,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,2023-03-24 20:00:00-07:00,2023-03-24


In [18]:
GCP_cost_table.loc[(GCP_cost_table['resource_name'].notnull())&(GCP_cost_table['resource_name'].str.contains('mongo-shard')),'component'] = 'Mongo DB Platform - Shard'
GCP_cost_table.loc[(GCP_cost_table['resource_name'].notnull())&(GCP_cost_table['resource_name'].str.contains('mongo-vm')),'component'] = 'Mongo DB Platform - Shard'
GCP_cost_table.loc[(GCP_cost_table['resource_name'].notnull())&(GCP_cost_table['resource_name'].str.contains('mongo-router')),'component'] = 'Mongo DB Platform - Router'
GCP_cost_table.loc[(GCP_cost_table['resource_name'].notnull())&(GCP_cost_table['resource_name'].str.contains('mongo-config')),'component'] = 'Mongo DB Platform - config'
GCP_cost_table.loc[(GCP_cost_table['resource_name'].notnull())&(GCP_cost_table['resource_name'].str.contains('mongo-analytics')),'component'] = 'Mongo DB Analytics'
GCP_cost_table.loc[(GCP_cost_table['resource_name'].notnull())&(GCP_cost_table['resource_name'].str.contains('mongo-temp')),'component'] = 'Mongo DB Platform - Temp'
GCP_cost_table.loc[(GCP_cost_table['resource_name'].notnull())&(GCP_cost_table['resource_name'].str.contains('kafka')),'component'] = 'Kafka Browser'
GCP_cost_table.loc[(GCP_cost_table['resource_name'].notnull())&(GCP_cost_table['resource_name'].str.contains('gke')),'component'] = 'Containers (gke)'
GCP_cost_table.loc[(GCP_cost_table['resource_name'].notnull())&(GCP_cost_table['resource_name'].str.contains('report')),'component'] = 'Reports'
GCP_cost_table.loc[(GCP_cost_table['resource_name'].notnull())&(GCP_cost_table['resource_name'].str.contains('nfs')),'component'] = 'NFS'
GCP_cost_table.loc[(GCP_cost_table['resource_name'].notnull())&(GCP_cost_table['resource_name'].str.contains('rabbit')),'component'] = 'Rabbit'
GCP_cost_table.loc[(GCP_cost_table['resource_name'].notnull())&(GCP_cost_table['resource_name'].str.contains('sftp')),'component'] = 'SFTP'
GCP_cost_table.loc[(GCP_cost_table['resource_name'].notnull())&(GCP_cost_table['resource_name'].str.contains('vpn')),'component'] = 'VPN'
GCP_cost_table.loc[(GCP_cost_table['resource_name'].notnull())&(GCP_cost_table['resource_name'].str.contains('manager')),'component'] = 'Manager'
GCP_cost_table.loc[(GCP_cost_table['resource_name'].notnull())&(GCP_cost_table['resource_name'].str.contains('postgres')),'component'] = 'Postgres'
GCP_cost_table.loc[(GCP_cost_table['resource_name'].notnull())&(GCP_cost_table['resource_name'].str.contains('powerbi')),'component'] = 'Power BI'
GCP_cost_table.loc[(GCP_cost_table['component'].isnull())&(GCP_cost_table['resource_global_name'].str.contains('scxyt1cloudsql')),'component'] = 'Cloud SQL'
GCP_cost_table.loc[(GCP_cost_table['component'].isnull())&(GCP_cost_table['resource_global_name'].str.contains('ytemyt1cloudsql')),'component'] = 'Cloud SQL'
GCP_cost_table.loc[(GCP_cost_table['component'].isnull())&(GCP_cost_table['resource_global_name'].str.contains('cloudsql')),'component'] = 'Cloud SQL'
GCP_cost_table.loc[(GCP_cost_table['component'].isnull())&(GCP_cost_table['resource_global_name'].str.contains('k8s-mojix-io')),'component'] = 'Containers (gke)'

In [19]:
GCP_cost_table.loc[(GCP_cost_table['component'].notnull())&(GCP_cost_table['resource_name'].str.contains('mongo-shard-1')),'SubComponent'] = 'Mongo DB Platform - Shard 1'
GCP_cost_table.loc[(GCP_cost_table['component'].notnull())&(GCP_cost_table['resource_name'].str.contains('mongo-shard-2')),'SubComponent'] = 'Mongo DB Platform - Shard 2'
GCP_cost_table.loc[(GCP_cost_table['component'].notnull())&(GCP_cost_table['resource_name'].str.contains('mongo-shard-3')),'SubComponent'] = 'Mongo DB Platform - Shard 3'
GCP_cost_table.loc[(GCP_cost_table['component'].notnull())&(GCP_cost_table['resource_name'].str.contains('mongo-shard-4')),'SubComponent'] = 'Mongo DB Platform - Shard 4'
GCP_cost_table.loc[(GCP_cost_table['component'].notnull())&(GCP_cost_table['resource_name'].str.contains('mongo-shard-5')),'SubComponent'] = 'Mongo DB Platform - Shard 5'
GCP_cost_table.loc[(GCP_cost_table['component'].notnull())&(GCP_cost_table['resource_name'].str.contains('mongo-shard-6')),'SubComponent'] = 'Mongo DB Platform - Shard 6'
GCP_cost_table.loc[(GCP_cost_table['component'].notnull())&(GCP_cost_table['resource_name'].str.contains('mongo-shard-7')),'SubComponent'] = 'Mongo DB Platform - Shard 7'
GCP_cost_table.loc[(GCP_cost_table['component'].notnull())&(GCP_cost_table['resource_name'].str.contains('mongo-vm002')),'SubComponent'] = 'Mongo DB Platform - Shard 0'
GCP_cost_table.loc[(GCP_cost_table['component'].notnull())&(GCP_cost_table['resource_name'].str.contains('mongo-vm001')),'SubComponent'] = 'Mongo DB Platform - Shard 0'
GCP_cost_table.loc[(GCP_cost_table['component'].notnull())&(GCP_cost_table['resource_name'].str.contains('mongo-vm003')),'SubComponent'] = 'Mongo DB Platform - Shard 0'

GCP_cost_table.loc[(GCP_cost_table['component'].notnull())&(GCP_cost_table['resource_name'].str.contains('mongo-router')),'SubComponent'] = 'Mongo DB Platform - Router'
GCP_cost_table.loc[(GCP_cost_table['component'].notnull())&(GCP_cost_table['resource_name'].str.contains('mongo-config')),'SubComponent'] = 'Mongo DB Platform - Config'
GCP_cost_table.loc[(GCP_cost_table['component'].notnull())&(GCP_cost_table['resource_name'].str.contains('mongo-analytics')),'SubComponent'] = 'Mongo DB Platform - Analytics'
GCP_cost_table.loc[(GCP_cost_table['component'].notnull())&(GCP_cost_table['resource_name'].str.contains('mongo-temp')),'SubComponent'] = 'Mongo DB Platform - Temp'

In [20]:
GCP_cost_table_f = GCP_cost_table[GCP_cost_table['component'].fillna('N/A').str.contains('Mongo')]

In [21]:
GCP_cost_table_f.tail()

,description_service,description_sku,usage_start_time,id_project,cost,usage_amount,usage_unit,credits,cost_type,resource_name,...,storage_cost,total_after_discounts,amount_credits,compute_discount,storage_discount,total_compute_after_discount,usage_start_time_LA,date_LA,component,SubComponent
512462,Compute Engine,Network Internet Egress from Americas to Austr...,2023-03-25 05:00:00+00:00,saas-mojixretail-io,0.000001,8856.0,bytes,,regular,mongo-vm001-centralus-saas,...,0.0,0.000001,0.0,0.0,0.0,0.000001,2023-03-24 22:00:00-07:00,2023-03-24,Mongo DB Platform - Shard,Mongo DB Platform - Shard 0
512463,Compute Engine,Network Internet Egress from Americas to Austr...,2023-03-25 06:00:00+00:00,saas-mojixretail-io,0.000000,144.0,bytes,,regular,mongo-vm001-centralus-saas,...,0.0,0.000000,0.0,0.0,0.0,0.000000,2023-03-24 23:00:00-07:00,2023-03-24,Mongo DB Platform - Shard,Mongo DB Platform - Shard 0
512464,Compute Engine,Network Internet Egress from Americas to Austr...,2023-03-25 07:00:00+00:00,saas-mojixretail-io,0.000001,4824.0,bytes,,regular,mongo-vm001-centralus-saas,...,0.0,0.000001,0.0,0.0,0.0,0.000001,2023-03-25 00:00:00-07:00,2023-03-25,Mongo DB Platform - Shard,Mongo DB Platform - Shard 0
512465,Compute Engine,Network Internet Egress from Americas to Austr...,2023-03-24 22:00:00+00:00,saas-mojixretail-io,0.000001,4994.0,bytes,,regular,mongo-vm003-centralus-saas,...,0.0,0.000001,0.0,0.0,0.0,0.000001,2023-03-24 15:00:00-07:00,2023-03-24,Mongo DB Platform - Shard,Mongo DB Platform - Shard 0
512466,Compute Engine,Network Internet Egress from Americas to Austr...,2023-03-24 23:00:00+00:00,saas-mojixretail-io,0.000001,4006.0,bytes,,regular,mongo-vm003-centralus-saas,...,0.0,0.000001,0.0,0.0,0.0,0.000001,2023-03-24 16:00:00-07:00,2023-03-24,Mongo DB Platform - Shard,Mongo DB Platform - Shard 0


In [22]:
df_cost_storage_gcp_agg = GCP_cost_table_f[GCP_cost_table_f['date_LA'].astype(str)==str_day_process].groupby(['date_LA','component','SubComponent']).sum().reset_index()

In [23]:
df_cost_storage_gcp_agg = df_cost_storage_gcp_agg[['date_LA','component','SubComponent','compute_cost','storage_cost','compute_discount','storage_discount','cost','amount_credits','total_after_discounts']]

In [0]:
#environment = 'YT1'

In [24]:
if (environment == 'YT1'):
    query = """
   SELECT * FROM `saas-analytics-io.raw.yt1_mongodb_db` WHERE date(date) = '%s'
   --SELECT * FROM `saas-analytics-io.raw.yt1_mongodb_collection` WHERE date(date) between '2022-07-01' and '2022-07-24'
  """%(str_day_process)

    query_job = client.query(
      query,
      location="us-central1",)

    print(query)
    df_mongo_db = query_job.to_dataframe()
    df_mongo_db.loc[:,'environment'] = environment
    df_mongo_db.head()
    
    
#if (environment == 'VT1'):
  #query = """
   #SELECT * FROM `saas-analytics-io.raw.vt1_mongodb_collection` WHERE date(date) = '%s'
   #--SELECT * FROM `saas-analytics-io.raw.vt1_mongodb_collection` WHERE date(date) = '2022-11-10'
  #"""%(str_day_process)

  #query_job = client.query(
   #   query,
    #  location="us-central1",)

  #print(query)
  #df_mongo = query_job.to_dataframe()
  #df_mongo.loc[:,'environment'] = environment
  #df_mongo.head()


   SELECT * FROM `saas-analytics-io.raw.yt1_mongodb_db` WHERE date(date) = '2023-03-24'
   --SELECT * FROM `saas-analytics-io.raw.yt1_mongodb_collection` WHERE date(date) between '2022-07-01' and '2022-07-24'
  


In [25]:
df_mongo_db.head()

,date,database_name,shard_id,shard,size_on_disk,empty,tenant,environment
0,2023-03-24 18:02:48.523797,base,rs0,0,2.170880e+05,False,BASE,YT1
1,2023-03-24 18:02:48.523797,config,rs0,0,4.813537e+08,False,CONFIG,YT1
2,2023-03-24 18:02:48.523797,config,config,0,5.612175e+08,False,CONFIG,YT1
3,2023-03-24 18:02:48.523797,viz_cdc,rs0,0,1.836745e+10,False,CDC,YT1
4,2023-03-24 18:02:48.523797,viz_celine,rs0,0,4.108832e+10,False,CELINE,YT1


In [26]:
df_mongo_db.loc[:,'date_LA'] = df_mongo_db['date'].dt.date
#df_mongo.loc[:,'date_normalized'] = str_day_process
#df_mongo['date_normalized'] = pd.to_datetime(df_mongo['date_normalized']).dt.date

In [27]:
df_mongo_db.loc[(df_mongo_db['shard_id'].str.contains('rs0')),'SubComponent'] = 'Mongo DB Platform - Shard 0'
df_mongo_db.loc[(df_mongo_db['shard_id'].str.contains('rs0_1')),'SubComponent'] = 'Mongo DB Platform - Shard 1'
df_mongo_db.loc[(df_mongo_db['shard_id'].str.contains('rs0_2')),'SubComponent'] = 'Mongo DB Platform - Shard 2'
df_mongo_db.loc[(df_mongo_db['shard_id'].str.contains('rs0_3')),'SubComponent'] = 'Mongo DB Platform - Shard 3'
df_mongo_db.loc[(df_mongo_db['shard_id'].str.contains('rs0_4')),'SubComponent'] = 'Mongo DB Platform - Shard 4'
df_mongo_db.loc[(df_mongo_db['shard_id'].str.contains('rs0_5')),'SubComponent'] = 'Mongo DB Platform - Shard 5'
df_mongo_db.loc[(df_mongo_db['shard_id'].str.contains('rs0_6')),'SubComponent'] = 'Mongo DB Platform - Shard 6'
df_mongo_db.loc[(df_mongo_db['shard_id'].str.contains('rs0_7')),'SubComponent'] = 'Mongo DB Platform - Shard 7'
df_mongo_db.loc[(df_mongo_db['shard_id'].str.contains('config')),'SubComponent'] = 'Mongo DB Platform - Config'

In [28]:
df_mongo_db.groupby('SubComponent').count()

,date,database_name,shard_id,shard,size_on_disk,empty,tenant,environment,date_LA
SubComponent,,,,,,,,,
Mongo DB Platform - Config,1,1,1,1,1,1,1,1,1
Mongo DB Platform - Shard 0,24,24,24,24,24,24,24,24,24
Mongo DB Platform - Shard 1,23,23,23,23,23,23,23,23,23
Mongo DB Platform - Shard 2,23,23,23,23,23,23,23,23,23
Mongo DB Platform - Shard 3,23,23,23,23,23,23,23,23,23
Mongo DB Platform - Shard 4,24,24,24,24,24,24,24,24,24
Mongo DB Platform - Shard 5,27,27,27,27,27,27,27,27,27
Mongo DB Platform - Shard 6,24,24,24,24,24,24,24,24,24
Mongo DB Platform - Shard 7,23,23,23,23,23,23,23,23,23


In [29]:
df_mongo_agg_1 = df_mongo_db.groupby(['date_LA','SubComponent','shard_id','environment']).sum()[['size_on_disk']].reset_index()
df_mongo_agg_2 = df_mongo_db.groupby(['date_LA','SubComponent','shard_id','tenant','environment']).sum()[['size_on_disk']].reset_index()

In [30]:
df_mongo_agg_1.tail()

,date_LA,SubComponent,shard_id,environment,size_on_disk
4,2023-03-24,Mongo DB Platform - Shard 3,rs0_3,YT1,3.502892e+12
5,2023-03-24,Mongo DB Platform - Shard 4,rs0_4,YT1,3.345446e+12
6,2023-03-24,Mongo DB Platform - Shard 5,rs0_5,YT1,3.325818e+12
7,2023-03-24,Mongo DB Platform - Shard 6,rs0_6,YT1,3.349032e+12
8,2023-03-24,Mongo DB Platform - Shard 7,rs0_7,YT1,3.436662e+12


In [31]:
df_agg_res_1 = pd.merge(df_mongo_agg_1, df_cost_storage_gcp_agg, how= 'outer', on =['date_LA','SubComponent'])

In [32]:
df_agg_res_1.loc[:,'compute_cost_by_byte'] = df_agg_res_1['compute_cost']/df_agg_res_1['size_on_disk']
df_agg_res_1.loc[:,'storage_cost_by_byte'] = df_agg_res_1['storage_cost']/df_agg_res_1['size_on_disk']
df_agg_res_1.loc[:,'compute_discount_by_byte'] = df_agg_res_1['compute_discount']/df_agg_res_1['size_on_disk']
df_agg_res_1.loc[:,'storage_discount_by_byte'] = df_agg_res_1['storage_discount']/df_agg_res_1['size_on_disk']
df_agg_res_1.loc[:,'cost_by_byte'] = df_agg_res_1['cost']/df_agg_res_1['size_on_disk']
df_agg_res_1.loc[:,'amount_credits_by_byte'] = df_agg_res_1['amount_credits']/df_agg_res_1['size_on_disk']
df_agg_res_1.loc[:,'total_after_discounts_by_byte'] = df_agg_res_1['total_after_discounts']/df_agg_res_1['size_on_disk']


In [33]:
df_agg_res_1 = df_agg_res_1[['date_LA','SubComponent','component',
             'compute_cost_by_byte','storage_cost_by_byte','compute_discount_by_byte','storage_discount_by_byte',
            'cost_by_byte','amount_credits_by_byte','total_after_discounts_by_byte','cost','total_after_discounts'
                             ,'compute_discount','storage_discount','compute_cost','storage_cost']]

In [34]:
df_results = pd.merge(df_mongo_agg_2, df_agg_res_1, how ='outer', on =['date_LA','SubComponent'])

In [35]:
df_results.tail()

,date_LA,SubComponent,shard_id,tenant,environment,size_on_disk,component,compute_cost_by_byte,storage_cost_by_byte,compute_discount_by_byte,storage_discount_by_byte,cost_by_byte,amount_credits_by_byte,total_after_discounts_by_byte,cost,total_after_discounts,compute_discount,storage_discount,compute_cost,storage_cost
189,2023-03-24,Mongo DB Platform - Shard 7,rs0_7,SHOW,YT1,1.704894e+09,Mongo DB Platform - Shard,9.835124e-12,1.089730e-11,-2.420210e-12,0.0,2.073242e-11,-2.420210e-12,1.831221e-11,71.250328,62.932886,-8.317442,0.0,33.799995,37.450333
190,2023-03-24,Mongo DB Platform - Shard 7,rs0_7,TOSCA,YT1,4.675519e+10,Mongo DB Platform - Shard,9.835124e-12,1.089730e-11,-2.420210e-12,0.0,2.073242e-11,-2.420210e-12,1.831221e-11,71.250328,62.932886,-8.317442,0.0,33.799995,37.450333
191,2023-03-24,Mongo DB Platform - Shard 7,rs0_7,TRR,YT1,7.494861e+07,Mongo DB Platform - Shard,9.835124e-12,1.089730e-11,-2.420210e-12,0.0,2.073242e-11,-2.420210e-12,1.831221e-11,71.250328,62.932886,-8.317442,0.0,33.799995,37.450333
192,2023-03-24,Mongo DB Platform - Analytics,NaN,NaN,NaN,NaN,Mongo DB Analytics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.490113,66.651415,-12.838698,0.0,46.542601,32.947512
193,2023-03-24,Mongo DB Platform - Router,NaN,NaN,NaN,NaN,Mongo DB Platform - Router,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.880592,33.620003,-12.260589,0.0,45.386376,0.494216


In [36]:
df_results.loc[:,'compute_cost_after_discount']= df_results['size_on_disk']*(df_results['compute_cost_by_byte']+df_results['compute_discount_by_byte'])
df_results.loc[:,'storage_cost_after_discount']= df_results['size_on_disk']*(df_results['storage_cost_by_byte']+df_results['storage_discount_by_byte'])
df_results.loc[:,'total_allocated_cost_after_discount']= df_results['size_on_disk']*df_results['total_after_discounts_by_byte']
df_results.loc[:,'total_allocated_cost']= df_results['size_on_disk']*df_results['cost_by_byte']

df_results.loc[df_results['component'].str.contains('Analytics'),'tenant']= 'internal'
df_results.loc[df_results['component'].str.contains('Analytics'),'compute_cost_after_discount']= df_results['compute_cost']+df_results['compute_discount']
df_results.loc[df_results['component'].str.contains('Analytics'),'storage_cost_after_discount']= df_results['storage_cost']+df_results['storage_discount']
df_results.loc[df_results['component'].str.contains('Analytics'),'total_allocated_cost_after_discount']= df_results['total_after_discounts']
df_results.loc[df_results['component'].str.contains('Analytics'),'total_allocated_cost']= df_results['cost']

df_results.loc[df_results['component'].str.contains('Router'),'tenant']= 'internal'
df_results.loc[df_results['component'].str.contains('Router'),'compute_cost_after_discount']= df_results['compute_cost']+df_results['compute_discount']
df_results.loc[df_results['component'].str.contains('Router'),'storage_cost_after_discount']= df_results['storage_cost']+df_results['storage_discount']
df_results.loc[df_results['component'].str.contains('Router'),'total_allocated_cost_after_discount']= df_results['total_after_discounts']
df_results.loc[df_results['component'].str.contains('Router'),'total_allocated_cost']= df_results['cost']

df_results.loc[df_results['component'].str.contains('Temp'),'tenant']= 'internal'
df_results.loc[df_results['component'].str.contains('Temp'),'compute_cost_after_discount']= df_results['compute_cost']+df_results['compute_discount']
df_results.loc[df_results['component'].str.contains('Temp'),'storage_cost_after_discount']= df_results['storage_cost']+df_results['storage_discount']
df_results.loc[df_results['component'].str.contains('Temp'),'total_allocated_cost_after_discount']= df_results['total_after_discounts']
df_results.loc[df_results['component'].str.contains('Temp'),'total_allocated_cost']= df_results['cost']

In [37]:
df_results

,date_LA,SubComponent,shard_id,tenant,environment,size_on_disk,component,compute_cost_by_byte,storage_cost_by_byte,compute_discount_by_byte,...,cost,total_after_discounts,compute_discount,storage_discount,compute_cost,storage_cost,compute_cost_after_discount,storage_cost_after_discount,total_allocated_cost_after_discount,total_allocated_cost
0,2023-03-24,Mongo DB Platform - Config,config,CONFIG,YT1,5.612175e+08,Mongo DB Platform - config,1.336836e-08,5.870754e-10,-4.096214e-09,...,7.832033,5.533166,-2.298867,0.0,7.502556,0.329477,5.203689,0.329477,5.533166,7.832033
1,2023-03-24,Mongo DB Platform - Shard 0,rs0,BASE,YT1,2.170880e+05,Mongo DB Platform - Shard,9.127541e-12,1.096407e-11,-2.206861e-12,...,75.723254,67.405817,-8.317437,0.0,34.400787,41.322467,0.000002,0.000002,0.000004,0.000004
2,2023-03-24,Mongo DB Platform - Shard 0,rs0,CDC,YT1,1.836745e+10,Mongo DB Platform - Shard,9.127541e-12,1.096407e-11,-2.206861e-12,...,75.723254,67.405817,-8.317437,0.0,34.400787,41.322467,0.127115,0.201382,0.328497,0.369032
3,2023-03-24,Mongo DB Platform - Shard 0,rs0,CELINE,YT1,4.108832e+10,Mongo DB Platform - Shard,9.127541e-12,1.096407e-11,-2.206861e-12,...,75.723254,67.405817,-8.317437,0.0,34.400787,41.322467,0.284359,0.450495,0.734854,0.825530
4,2023-03-24,Mongo DB Platform - Shard 0,rs0,CENT,YT1,1.885928e+12,Mongo DB Platform - Shard,9.127541e-12,1.096407e-11,-2.206861e-12,...,75.723254,67.405817,-8.317437,0.0,34.400787,41.322467,13.051908,20.677445,33.729353,37.891334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,2023-03-24,Mongo DB Platform - Shard 7,rs0_7,SHOW,YT1,1.704894e+09,Mongo DB Platform - Shard,9.835124e-12,1.089730e-11,-2.420210e-12,...,71.250328,62.932886,-8.317442,0.0,33.799995,37.450333,0.012642,0.018579,0.031220,0.035347
190,2023-03-24,Mongo DB Platform - Shard 7,rs0_7,TOSCA,YT1,4.675519e+10,Mongo DB Platform - Shard,9.835124e-12,1.089730e-11,-2.420210e-12,...,71.250328,62.932886,-8.317442,0.0,33.799995,37.450333,0.346686,0.509505,0.856191,0.969348
191,2023-03-24,Mongo DB Platform - Shard 7,rs0_7,TRR,YT1,7.494861e+07,Mongo DB Platform - Shard,9.835124e-12,1.089730e-11,-2.420210e-12,...,71.250328,62.932886,-8.317442,0.0,33.799995,37.450333,0.000556,0.000817,0.001372,0.001554
192,2023-03-24,Mongo DB Platform - Analytics,NaN,internal,NaN,NaN,Mongo DB Analytics,NaN,NaN,NaN,...,79.490113,66.651415,-12.838698,0.0,46.542601,32.947512,33.703903,32.947512,66.651415,79.490113


In [38]:
df_results_upload = df_results[['date_LA','SubComponent','shard_id','tenant','environment','size_on_disk','component',
                                'total_after_discounts','compute_cost_after_discount','storage_cost_after_discount','total_allocated_cost_after_discount','total_allocated_cost']]

In [39]:
df_results_upload.tail()

,date_LA,SubComponent,shard_id,tenant,environment,size_on_disk,component,total_after_discounts,compute_cost_after_discount,storage_cost_after_discount,total_allocated_cost_after_discount,total_allocated_cost
189,2023-03-24,Mongo DB Platform - Shard 7,rs0_7,SHOW,YT1,1.704894e+09,Mongo DB Platform - Shard,62.932886,0.012642,0.018579,0.031220,0.035347
190,2023-03-24,Mongo DB Platform - Shard 7,rs0_7,TOSCA,YT1,4.675519e+10,Mongo DB Platform - Shard,62.932886,0.346686,0.509505,0.856191,0.969348
191,2023-03-24,Mongo DB Platform - Shard 7,rs0_7,TRR,YT1,7.494861e+07,Mongo DB Platform - Shard,62.932886,0.000556,0.000817,0.001372,0.001554
192,2023-03-24,Mongo DB Platform - Analytics,NaN,internal,NaN,NaN,Mongo DB Analytics,66.651415,33.703903,32.947512,66.651415,79.490113
193,2023-03-24,Mongo DB Platform - Router,NaN,internal,NaN,NaN,Mongo DB Platform - Router,33.620003,33.125787,0.494216,33.620003,45.880592


In [43]:
table = "saas-analytics-io.processed.yt1_mongo_db_cost_allocation"
df_results_upload.to_gbq(table,  if_exists='append')

AttributeError: module 'google.auth.credentials' has no attribute 'CredentialsWithTokenUri'